# Intro to LLMstudio - proxy

This tutorial serves as an introduction to LLMstudio, a platform designed to facilitate interactions with large language models (LLMs) through a proxy server. By following this guide, users will learn how to set up and run a proxy server that acts as an intermediary between the client and the LLM provider, ensuring seamless communication and data handling. The notebook demonstrates the process of initializing the server, configuring the proxy, and making requests to the LLM, providing a comprehensive overview of the system's capabilities.

Additionally, the tutorial highlights the integration of tracking features within LLMstudio, allowing users to monitor and log interactions with the LLMs. This is particularly useful for analyzing performance metrics, such as latency and token usage, which can help in optimizing the model's responses and understanding its behavior in different scenarios. By the end of this tutorial, users will have a solid foundation in setting up and utilizing LLMstudio for efficient and effective LLM interactions.



## start proxy server


In [1]:
from llmstudio.server import start_servers

# default port is 50001. set the environment varible to specify which host and port; LLMSTUDIO_ENGINE_HOST, LLMSTUDIO_ENGINE_PORT
start_servers(proxy=True, tracker=False)

Proxy server already running on 0.0.0.0:8001


In [2]:
from llmstudio_proxy.provider import LLMProxyProvider as LLM
from llmstudio_proxy.provider import ProxyConfig

llm = LLM(provider="openai", 
          proxy_config=ProxyConfig(host="0.0.0.0", port="8001"))

Connected to LLMStudio Proxy @ 0.0.0.0:8001


In [3]:
result = llm.chat("olá", model="gpt-4o")


In [4]:
result.chat_output, result.metrics

('Olá! Como posso ajudá-lo hoje?',
 {'input_tokens': 2,
  'output_tokens': 11,
  'total_tokens': 13,
  'cost_usd': 0.000175,
  'latency_s': 0.8093140125274658,
  'time_to_first_token_s': 0.541956901550293,
  'inter_token_latency_s': 0.028835773468017578,
  'tokens_per_second': 12.356143406896255})

### Use the LLMStudio SDK entrypoint


In [5]:
from llmstudio.providers import LLM
from pprint import pprint

In [12]:
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
from llmstudio_proxy.provider import ProxyConfig
proxy = ProxyConfig(host="0.0.0.0", port="8001")
print(proxy)

openai = LLM("openai", proxy_config=proxy)


host='0.0.0.0' port='8001' url=None username=None password=None
Connected to LLMStudio Proxy @ 0.0.0.0:8001


In [13]:
openai._provider


### Chat (non-stream)

In [14]:
openai.chat("What's your name", model="gpt-4o")

ChatCompletion(id='fb38ddcd-d53d-464a-af4d-99de07cb4659', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I’m called ChatGPT! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729702067, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output='I’m called ChatGPT! How can I assist you today?', chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729702067.576836, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 14, 'total_tokens': 18, 'cost_usd': 0.00023, 'latency_s': 0.6081359386444092, 'time_to_first_token_s': 0.44050097465515137, 'inter_token_latency_s': 0.011918153081621443, 'tokens_per_second': 24.665537829315557})

#### Async version

In [19]:
result = await openai.achat("What's your name", model="gpt-4o")
pprint(result)

None


### Chat (stream)

In [16]:
response = openai.chat("Write a paragfraph about space", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, the vast and enigmatic expanse beyond Earth's atmosphere, has captivated human imagination for centuries.

 It is a realm of infinite possibilities, where stars twinkle like distant beacons and galaxies swirl in

 cosmic dances. Within this boundless void lie the mysteries of black holes, neutron stars, and dark

 matter, challenging our understanding of physics and the universe itself. Space exploration, driven by a desire to

 explore and learn, has led to remarkable achievements, from landing on the moon to sending probes to distant

 planets and beyond. As technology advances, our ability to explore and understand space grows, promising new insights

 into the origins of our universe and the potential for life beyond our planet. The final frontier continues to

 inspire and beckon, reminding us of our small but significant place in the cosmos.

## Metrics:
{'cost_usd': 0.002425,
 'input_tokens': 8,
 'inter_token_latency_s': 0.015780606846900504,
 'latency_s': 3.44837594032

#### Async version

In [18]:
i=0
async for chunk in await openai.achat("Write a paragfraph about space", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




Space is a vast and enigmatic expanse that extends beyond the confines of our planet, holding the

 mysteries of the universe within its dark, star-studded fabric. It is where galaxies collide,

 stars are born in stellar nurseries, and black holes warp the very fabric of time and space.

 The sheer scale of space is beyond human comprehension; it encompasses billions of galaxies, each containing millions or

 even billions of stars, along with myriad planets, moons, and celestial phenomena. As humanity ventures further

 into this cosmic wilderness, from the Moon to Mars and beyond, we continue to unravel the secrets of

 our origins and the potential for life beyond Earth. The exploration of space not only fuels our curiosity but

 also challenges our understanding of physics, chemistry, and the fundamental laws that govern the universe, inviting us

 to ponder our place in the grandeur of existence.

## Metrics:
{'cost_usd': 0.0001032,
 'input_tokens': 8,
 'inter_token_latency_s':